# Importing Dataset


In [7]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
df_resume=pd.read_csv('raw_data/Data_Set_CV_Cleaned_version test.csv')
df_resume.drop(['Unnamed: 0'],axis=1,inplace=True)
df_resume=df_resume.rename(columns={'New_Category':'Category','CV_cleaned':'Resume'})
df_resume.dropna(inplace=True)
df_resume=df_resume.drop_duplicates()
df_resume['lenght']=df_resume['Resume'].apply(len)

In [42]:
len_max=10000
df_resume_filtred=df_resume[df_resume['lenght']<len_max]

In [43]:
num_categories= len(df_resume_filtred['Category'].value_counts())
num_categories

15

In [44]:
from sklearn.preprocessing import LabelEncoder
# Convert labels to integers
label_encoder = LabelEncoder()
df_resume_filtred['N_Category'] = label_encoder.fit_transform(df_resume_filtred['Category'])

/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_7524/2083349867.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resume_filtred['N_Category'] = label_encoder.fit_transform(y)


# Pre Train model : BERT

In [45]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import legacy


## Charger le modèle BERT


In [46]:
# model_name = "bert-base-uncased"
model_name = "bert-large-uncased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
#Division des données en ensembles d'entraînement, de validation et de test
# Création d'un échantillon aléatoire de  du dataset complet
frac=0.2
sample_df = df_resume_filtred.sample(frac=frac, random_state=42)
full_df=df_resume_filtred


df=sample_df

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [25]:
#Tokenisation avec BERT

# Charger le tokenizer BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Tokeniser les textes d'entraînement, de validation et de test
max_length=100

train_tokens = tokenizer(list(train_data['Resume']), truncation=True, padding=True, max_length=max_length, return_tensors="tf")
val_tokens = tokenizer(list(val_data['Resume']), truncation=True, padding=True, max_length=max_length, return_tensors="tf")
test_tokens = tokenizer(list(test_data['Resume']), truncation=True, padding=True, max_length=max_length, return_tensors="tf")

# Extraire les input_ids et attention_masks
train_input_ids, train_attention_masks = train_tokens['input_ids'], train_tokens['attention_mask']
val_input_ids, val_attention_masks = val_tokens['input_ids'], val_tokens['attention_mask']
test_input_ids, test_attention_masks = test_tokens['input_ids'], test_tokens['attention_mask']


In [38]:
# Modifier la structure de votre modèle en utilisant le modèle fonctionnel
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")

# Incorporer BERT dans votre modèle
bert_output = model({'input_ids': input_ids, 'attention_mask': attention_mask}).logits

# Ajouter la couche Dense pour la classification
output = tf.keras.layers.Dense(units=len(df['N_Category'].unique()), activation='softmax')(bert_output)

# Créer le modèle fonctionnel
classification_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Définir le learning rate initial et la taille du batch
initial_learning_rate = 1e-2
batch_size = 32

# Définir le plan d'apprentissage (learning schedule) avec une décroissance exponentielle
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=len(train_data) // batch_size, decay_rate=0.9, staircase=True
)


KeyError: 'N_Category'

In [27]:
# Compiler le modèle avec l'optimiseur utilisant le plan d'apprentissage
classification_model.compile(
    # optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    optimizer = legacy.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [28]:
# Définir l'arrêt précoce
early_stopping = EarlyStopping(
    monitor='val_loss',  # Surveiller la perte sur l'ensemble de validation
    patience=2,           # Nombre d'époques sans amélioration avant l'arrêt
    restore_best_weights=True  # Restaurer les meilleurs poids du modèle
)

In [29]:
history = classification_model.fit(
    {'input_ids': train_input_ids, 'attention_mask': train_attention_masks},
    train_data['Category'].values,
    epochs=1,
    batch_size=32,
    validation_data=({'input_ids': val_input_ids, 'attention_mask': val_attention_masks}, val_data['Category'].values)
)

2024-01-23 19:34:36.743429: W tensorflow/core/framework/op_kernel.cc:1816] OP_REQUIRES failed at cast_op.cc:122 : UNIMPLEMENTED: Cast string to int64 is not supported


UnimplementedError: Graph execution error:

Detected at node sparse_categorical_crossentropy/Cast defined at (most recent call last):
  File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code

  File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/lib/p

In [ ]:
# Évaluer le modèle
test_metrics = classification_model.evaluate(
    {'input_ids': test_input_ids, 'attention_mask': test_attention_masks},
    test_data['Category'].values
)

print("Test Accuracy:", test_metrics[1])